In [1]:
import numpy as np
import pandas as pd
import os
import sys
import re
from sklearn import tree

In [312]:
# Importing all the models that I am going to use on my dataset
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
#Feature selection
from sklearn.feature_selection import RFE
from sklearn.ensemble import ExtraTreesClassifier

# Data Preprocessing
In this section , I start with Final.csc which is the final csv formed after merging all the other csv's with the MillionSongDataset. For more information in detail about how this happened, please check -------

In [488]:
data=pd.read_csv('Final.csv',sep=',')
D1=data.copy()
# data = data.drop(['Unnamed: 0','song_keys','song_bars','song_beats','song_modes','song_tatum','song_time_signatures','artist_names','song_ids','song_titles','track_id','lyrics','artistlocation','user_id','genre','song_realease_years'], 1)
data = data.drop(['Unnamed: 0'],1)
print(len(data))
print(data.shape)
data.head(2)

6384
(6384, 22)


,artist_familiarty,artist_hotness,artist_names,song_bars,song_beats,song_durations,song_hotness,song_ids,song_keys,song_modes,...,song_tempo,song_time_signatures,song_titles,track_id,genre,lyrics,artistlocation,user_id,play_count,rating
0,0.733433,0.455559,Shadows Fall,[ 1.35712 4.17899 7.00987 9.83158 ...,[ 0.65324 1.35712 2.06418 2.76881 ...,283.48036,0.687874,SOKMPKV12A67AE241B,0,0,...,84.992,4,What Drives The Weak,TRARUOP12903CF2384,Metal,the|19 you|3 to|6 a|4 me|1 it|4 not|1 in|1 is|...,Boston,af031a4432b7a31431231a28459b04affbd0014e,1,1
1,0.733433,0.455559,Shadows Fall,[ 1.35712 4.17899 7.00987 9.83158 ...,[ 0.65324 1.35712 2.06418 2.76881 ...,283.48036,0.687874,SOKMPKV12A67AE241B,0,0,...,84.992,4,What Drives The Weak,TRARUOP12903CF2384,Metal,the|19 you|3 to|6 a|4 me|1 it|4 not|1 in|1 is|...,Boston,6c618f60b9b27b2b951854c56a7c96991514ca4f,1,1


In [489]:
data['count'] = 1

In [490]:
median_song_hotness = data['song_hotness'].dropna().median()
data['song_hotness'] = data.song_hotness.mask(data.song_hotness == 0, median_song_hotness)

In [491]:
mean_artist_hotness = data['artist_hotness'].dropna().mean()
data['artist_hotness'] = data.artist_hotness.mask(data.artist_hotness == 0, mean_artist_hotness)
mean_artist_familiarty = data['artist_familiarty'].dropna().mean()

In [492]:
data['song_bars'].head()

0    [   1.35712    4.17899    7.00987    9.83158  ...
1    [   1.35712    4.17899    7.00987    9.83158  ...
2    [   1.35712    4.17899    7.00987    9.83158  ...
3    [   1.35712    4.17899    7.00987    9.83158  ...
4    [   1.35712    4.17899    7.00987    9.83158  ...
Name: song_bars, dtype: object

In [493]:
data['beats_number'] = [len(x) for x in data.song_beats]
data['bars_number'] = [len(x) for x in data.song_bars]
data['tatum_number'] = [len(x) for x in data.song_tatum]

In [494]:
data['log_play_count_sum'] = np.log(data[['play_count']])

In [495]:
data.head(2)

,artist_familiarty,artist_hotness,artist_names,song_bars,song_beats,song_durations,song_hotness,song_ids,song_keys,song_modes,...,lyrics,artistlocation,user_id,play_count,rating,count,beats_number,bars_number,tatum_number,log_play_count_sum
0,0.733433,0.455559,Shadows Fall,[ 1.35712 4.17899 7.00987 9.83158 ...,[ 0.65324 1.35712 2.06418 2.76881 ...,283.48036,0.687874,SOKMPKV12A67AE241B,0,0,...,the|19 you|3 to|6 a|4 me|1 it|4 not|1 in|1 is|...,Boston,af031a4432b7a31431231a28459b04affbd0014e,1,1,1,4439,1105,109,0.0
1,0.733433,0.455559,Shadows Fall,[ 1.35712 4.17899 7.00987 9.83158 ...,[ 0.65324 1.35712 2.06418 2.76881 ...,283.48036,0.687874,SOKMPKV12A67AE241B,0,0,...,the|19 you|3 to|6 a|4 me|1 it|4 not|1 in|1 is|...,Boston,6c618f60b9b27b2b951854c56a7c96991514ca4f,1,1,1,4439,1105,109,0.0


# 
Dropping all the unwanted features that are not required for this section.

In [496]:
data = data.drop(['artist_names','song_ids','lyrics','song_tatum','song_bars','song_beats','artistlocation','user_id','genre','track_id','song_titles','count'], 1)
data.head(1)

,artist_familiarty,artist_hotness,song_durations,song_hotness,song_keys,song_modes,song_realease_years,song_tempo,song_time_signatures,play_count,rating,beats_number,bars_number,tatum_number,log_play_count_sum
0,0.733433,0.455559,283.48036,0.687874,0,0,2004,84.992,4,1,1,4439,1105,109,0.0


In [497]:
# I am going to use song_hotness as my target while training and tesing the model
y = np.asarray(data['song_hotness'], dtype="|S6")

In [498]:
# splitting the train data
X=data[['artist_familiarty','artist_hotness','song_durations',\
        'song_keys','song_modes','song_realease_years','song_tempo','song_time_signatures','play_count','beats_number','bars_number','tatum_number','log_play_count_sum','rating','song_hotness']]

In [499]:
X.head(2)

,artist_familiarty,artist_hotness,song_durations,song_keys,song_modes,song_realease_years,song_tempo,song_time_signatures,play_count,beats_number,bars_number,tatum_number,log_play_count_sum,rating,song_hotness
0,0.733433,0.455559,283.48036,0,0,2004,84.992,4,1,4439,1105,109,0.0,1,0.687874
1,0.733433,0.455559,283.48036,0,0,2004,84.992,4,1,4439,1105,109,0.0,1,0.687874


In [500]:
# showing what the dataframe actually contains
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6384 entries, 0 to 6383
Data columns (total 15 columns):
artist_familiarty       6384 non-null float64
artist_hotness          6384 non-null float64
song_durations          6384 non-null float64
song_keys               6384 non-null int64
song_modes              6384 non-null int64
song_realease_years     6384 non-null int64
song_tempo              6384 non-null float64
song_time_signatures    6384 non-null int64
play_count              6384 non-null int64
beats_number            6384 non-null int64
bars_number             6384 non-null int64
tatum_number            6384 non-null int64
log_play_count_sum      6384 non-null float64
rating                  6384 non-null int64
song_hotness            6384 non-null float64
dtypes: float64(6), int64(9)
memory usage: 748.2 KB


In [501]:
#Train-test split on X and y
X_train, X_test, y_train, y_test = train_test_split (X,y,train_size=.7,random_state=42)

/Users/shomronjacob/anaconda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2010: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [502]:
# Printing all unique features of the dataframe
for c in data.columns:
    print ("%s has %s unique categories" % (c, data[c].nunique()))

artist_familiarty has 364 unique categories
artist_hotness has 369 unique categories
song_durations has 486 unique categories
song_hotness has 399 unique categories
song_keys has 12 unique categories
song_modes has 2 unique categories
song_realease_years has 44 unique categories
song_tempo has 500 unique categories
song_time_signatures has 5 unique categories
play_count has 56 unique categories
rating has 5 unique categories
beats_number has 460 unique categories
bars_number has 392 unique categories
tatum_number has 221 unique categories
log_play_count_sum has 56 unique categories


# Feature Selection
In this section I will first apply few supervised learning algorithms on my data to check if the accuracy of the model is good or bad and will also select the important features from the given list of 23 features (before dropping the unwanted ones) using RFE/ Recursive Feature Elimination.

# 1. Logistic Regression and RFE

Recursive Feature Elimination (RFE is a feature selection approach. It recursively removes attributes and builds a model on those attributes that remain. It uses the model accuracy to identify which attributes (and combination of attributes) contribute the most to predicting the target attribute.

In [503]:
LRM=LogisticRegression()
rfe= RFE(LRM, 5)
rfe= rfe.fit(X_train, y_train)
print("test accuracy is-",rfe.score(X_test,y_test))
print("train score is-",rfe.score(X_train,y_train))

test accuracy is- 0.263048016701
train score is- 0.245971351835


In [504]:
rfe_support=pd.DataFrame(rfe.support_, index = X_train.columns.values, columns =['support'])
rfe_support

,support
artist_familiarty,True
artist_hotness,True
song_durations,False
song_keys,False
song_modes,True
song_realease_years,False
song_tempo,False
song_time_signatures,False
play_count,False
beats_number,False


In [505]:
rfe_ranking=pd.DataFrame(rfe.ranking_, index = X_train.columns.values, columns =['ranking']).sort_values('ranking',ascending=[True])
rfe_ranking

,ranking
artist_familiarty,1
artist_hotness,1
song_modes,1
rating,1
song_hotness,1
song_time_signatures,2
song_keys,3
play_count,4
log_play_count_sum,5
tatum_number,6


# Discussion
From the above result, it seems that the relevant features are artist_familiarty, artist_hotness, song_modes, song_hotness and rating.

# Extra tree classifier
"An extra trees classifier, otherwise known as an “Extremely randomized trees” classifier, is a variant of a random forest. Unlike a random forest, at each step the entire sample is used and decision boundaries are picked at random, rather than the best one"-wiki

In [506]:
etc_model = ExtraTreesClassifier()
etc_model.fit(X_train, y_train)
etc_df = pd.DataFrame(etc_model.feature_importances_, index = X_train.columns.values, columns =['importance'])
etc_df.sort_values('importance',ascending=[False])

,importance
artist_hotness,0.113387
tatum_number,0.103906
beats_number,0.095421
song_tempo,0.094700
song_hotness,0.094672
song_keys,0.093077
song_durations,0.091319
bars_number,0.089033
song_realease_years,0.072586
artist_familiarty,0.070616


In [507]:
rfc_model=RandomForestClassifier()
rfc_model.fit(X_train, y_train)
rfc_df = pd.DataFrame(rfc_model.feature_importances_, index = X_train.columns.values, columns =['importance'])
rfc_df.sort_values('importance',ascending=[False])

,importance
song_hotness,0.155628
artist_hotness,0.110407
song_realease_years,0.109590
artist_familiarty,0.102658
bars_number,0.090647
song_durations,0.089564
beats_number,0.086979
song_tempo,0.081801
tatum_number,0.063560
song_keys,0.056548


In [508]:
rfe2 = RFE(rfc_model, 5)
rfe2 = rfe2.fit(X_train, y_train)
print("The train score for RFE is-",rfe2.score(X_train, y_train))

The train score for RFE is- 0.999328558639


In [509]:
rfe2_support=pd.DataFrame(rfe2.support_, index = X_train.columns.values, columns =['support'])
rfe2_support

,support
artist_familiarty,True
artist_hotness,True
song_durations,False
song_keys,False
song_modes,False
song_realease_years,True
song_tempo,True
song_time_signatures,False
play_count,False
beats_number,False


# Dicussion
According to the Feature Importance method, artist_hotness, artist_familiarty, bars_number,song_hotness and song_tempo are important.
So our model has to use artist_hotness,song_hotness and artist_familiarty which are the most common ones.

This also forms the Transparent part of this project.The results obtained after applying RFE or Extra Tree classifer for feature selection help in identifying which features to use and which one to leave.


In [510]:
data.corr()["song_hotness"]

artist_familiarty       0.319904
artist_hotness          0.229750
song_durations         -0.127648
song_hotness            1.000000
song_keys              -0.144422
song_modes              0.171968
song_realease_years     0.111228
song_tempo             -0.062757
song_time_signatures    0.121436
play_count             -0.011425
rating                  0.013509
beats_number           -0.204190
bars_number            -0.183177
tatum_number           -0.056421
log_play_count_sum      0.023159
Name: song_hotness, dtype: float64

In [511]:
data.shape

(6384, 15)

# Supervised Learning Models- using song_hotness
1. Linear Regression

In [512]:
X1_train = X_train[['artist_familiarty','artist_hotness','song_tempo','rating','play_count','song_durations','song_hotness']]
X1_test = X_test[['artist_familiarty','artist_hotness','song_tempo','rating','play_count','song_durations','song_hotness']]

In [513]:
# X1_train = X_train[['artist_familiarty','artist_hotness','beats_number','song_keys','song_modes','song_realease_years','song_time_signatures','bars_number','beats_number','log_play_count_sum','song_tempo','rating','play_count','tatum_number']]
# X1_test = X_test[['artist_familiarty','artist_hotness','beats_number','song_keys','song_modes','song_realease_years','song_time_signatures','bars_number','beats_number','log_play_count_sum','song_tempo','rating','play_count','tatum_number']]

In [514]:
X2_train = X_train[['artist_familiarty','artist_hotness','song_modes','song_time_signatures','log_play_count_sum','beats_number','song_keys','song_realease_years']]
X2_test = X_test[['artist_familiarty','artist_hotness','song_modes','song_time_signatures','log_play_count_sum','beats_number','song_keys','song_realease_years']]

In [515]:
LRM.fit(X1_train,y_train)
LRM.score(X1_test,y_test)

0.27139874739039666

In [516]:
LRM.fit(X2_train,y_train)
LRM.score(X2_test,y_test)

0.4034446764091858

# Note-
The accuracy of the model is increasing if we take into consideration the features that were deemed not important.

# 2. Decision Tree Classifier

In [517]:
DT = DecisionTreeClassifier()
DT.fit(X1_train,y_train)
DT.score(X1_test,y_test)

0.97077244258872653

In [518]:
DT.fit(X2_train,y_train)
DT.score(X2_test,y_test)

0.96868475991649272

# 
Overfitting issue encountered here!

# 3. Support vector machines

In [519]:
from sklearn import svm
svm_model = svm.SVC()

In [520]:
svm_model.fit(X1_train,y_train)
svm_model.score(X1_test,y_test)

0.90814196242171186

# 
Overfitting again!

# 4. Regulariazation

In [521]:
D1.head(1)

,Unnamed: 0,artist_familiarty,artist_hotness,artist_names,song_bars,song_beats,song_durations,song_hotness,song_ids,song_keys,...,song_tempo,song_time_signatures,song_titles,track_id,genre,lyrics,artistlocation,user_id,play_count,rating
0,0,0.733433,0.455559,Shadows Fall,[ 1.35712 4.17899 7.00987 9.83158 ...,[ 0.65324 1.35712 2.06418 2.76881 ...,283.48036,0.687874,SOKMPKV12A67AE241B,0,...,84.992,4,What Drives The Weak,TRARUOP12903CF2384,Metal,the|19 you|3 to|6 a|4 me|1 it|4 not|1 in|1 is|...,Boston,af031a4432b7a31431231a28459b04affbd0014e,1,1


In [522]:
D2=D1.copy()
D1 = D1.drop(['Unnamed: 0','artist_names','song_ids','song_tatum','song_bars','song_beats','artistlocation','user_id','track_id','song_titles'], 1)

In [523]:
#for categorical features
from patsy import dmatrices
y, X = dmatrices('song_hotness ~ artist_familiarty + artist_hotness', data=D1, return_type='dataframe')

In [525]:
lr1 = LinearRegression()
lr1.fit(X, y).score(X, y)

0.038387594574330053

In [526]:
from sklearn.preprocessing import OneHotEncoder
catty = D1[['genre']]
enc = OneHotEncoder(categorical_features=catty,handle_unknown='error', n_values='auto', sparse=True)
print(enc)

OneHotEncoder(categorical_features=         genre
0        Metal
1        Metal
2        Metal
3        Metal
4        Metal
5         Rock
6         Rock
7         Rock
8         Rock
9         Rock
10        Rock
11        Rock
12        Rock
13        Rock
14        Rock
15        Rock
16        Rock
17       ...
6379       Rap
6380       Rap
6381       Rap
6382       Rap
6383       Rap

[6384 rows x 1 columns],
       dtype=<class 'numpy.float64'>, handle_unknown='error',
       n_values='auto', sparse=True)


In [530]:
y, X = dmatrices('song_hotness ~ artist_familiarty + artist_hotness + genre', data=D1, return_type='dataframe')

In [531]:
lr1 = LinearRegression()
lr1.fit(X, y).score(X, y)

0.1619503419455891

# 
Genre addition doesnt really help our accuracy that much. Hence, all the more reason why I will shift to unsupervised learning methods.

This part is transparent enough to anyone reading this notebook. It helps in understanding how I selected the features and why there is a need for Unsupervised Learning algorithms.

# Unsupervised Learning
The data used for unsupervised learning was preprocessed using the inital csv file called Final.csv and saved as data_USL.csv. For more information on how this happened, check ----

In [528]:
df= pd.read_csv('data_USL.csv', sep=',')
df.shape

(6384, 18)

In [529]:
# D1 = D1.drop(['Unnamed: 0',1])
df.head(1)


,Unnamed: 0,artist_familiarty,artist_hotness,artist_names,song_durations,song_hotness,song_ids,song_titles,rating,track_id,genre,artistlocation,user_id,play_count,song_tempo,song_time_signatures,lyrics,index
0,0,0.733433,0.455559,Shadows Fall,283.48036,0.687874,SOKMPKV12A67AE241B,What Drives The Weak,1,TRARUOP12903CF2384,Metal,Boston,af031a4432b7a31431231a28459b04affbd0014e,1,84.992,4,the|19 you|3 to|6 a|4 me|1 it|4 not|1 in|1 is|...,1


In [532]:
song_names = df.groupby('song_ids').song_titles.first()
song_df=df.groupby('song_ids')['song_titles','artist_names'].first()

In [533]:
song_df.head(2)

,song_titles,artist_names
song_ids,,
SOAAEHR12A6D4FB060,Slaves & Bulldozers,Soundgarden
SOABRXK12A8C130A36,Saturday Love,Alexander ONeal


In [534]:
df.rating.value_counts().sort_index()

1    4783
2    1332
3     188
4      55
5      26
Name: rating, dtype: int64

In [535]:
#first we nee to vectorize the genre category from my dataframe
from sklearn.feature_extraction import DictVectorizer
categorical_features = df[['genre']]

In [536]:
dv = DictVectorizer()

In [537]:
cat_matrix = dv.fit_transform(categorical_features.T.to_dict().values())
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6384 entries, 0 to 6383
Data columns (total 18 columns):
Unnamed: 0              6384 non-null int64
artist_familiarty       6384 non-null float64
artist_hotness          6384 non-null float64
artist_names            6384 non-null object
song_durations          6384 non-null float64
song_hotness            6384 non-null float64
song_ids                6384 non-null object
song_titles             6384 non-null object
rating                  6384 non-null int64
track_id                6384 non-null object
genre                   6384 non-null object
artistlocation          6384 non-null object
user_id                 6384 non-null object
play_count              6384 non-null int64
song_tempo              6384 non-null float64
song_time_signatures    6384 non-null int64
lyrics                  6384 non-null object
index                   6384 non-null int64
dtypes: float64(5), int64(5), object(8)
memory usage: 897.8+ KB


In [538]:
for c in df.columns:
    print ("%s has %s unique categories" % (c, df[c].nunique()))

Unnamed: 0 has 6384 unique categories
artist_familiarty has 364 unique categories
artist_hotness has 369 unique categories
artist_names has 368 unique categories
song_durations has 486 unique categories
song_hotness has 400 unique categories
song_ids has 500 unique categories
song_titles has 500 unique categories
rating has 5 unique categories
track_id has 500 unique categories
genre has 54 unique categories
artistlocation has 172 unique categories
user_id has 6057 unique categories
play_count has 56 unique categories
song_tempo has 500 unique categories
song_time_signatures has 5 unique categories
lyrics has 500 unique categories
index has 6384 unique categories


In [539]:
df['artist_familiarty'] = df.artist_familiarty.astype(str)
df['song_ids'] = df.song_ids.astype(str)

In [540]:
from patsy import dmatrix
X_features = dmatrix('artist_familiarty + song_ids', data=df.fillna(0), return_type='dataframe')

In [541]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=10)
X_genre = cv.fit_transform(df.genre)

In [542]:
from scipy.sparse import hstack

In [543]:
other_features = df[['artist_familiarty','artist_hotness','song_tempo','rating','play_count','song_durations','song_hotness']]

In [584]:
other_features

,artist_familiarty,artist_hotness,song_tempo,rating,play_count,song_durations,song_hotness
0,0.733433226583,0.455559,84.992,1,1,283.48036,0.687874
1,0.733433226583,0.455559,84.992,1,1,283.48036,0.687874
2,0.733433226583,0.455559,84.992,1,1,283.48036,0.687874
3,0.733433226583,0.455559,84.992,1,1,283.48036,0.687874
4,0.733433226583,0.455559,84.992,1,1,283.48036,0.687874
5,0.887386080596,0.791143,105.095,2,10,208.95302,0.872229
6,0.887386080596,0.791143,105.095,1,1,208.95302,0.872229
7,0.887386080596,0.791143,105.095,1,1,208.95302,0.872229
8,0.887386080596,0.791143,105.095,2,4,208.95302,0.872229
9,0.887386080596,0.791143,105.095,1,2,208.95302,0.872229


In [544]:
X = np.hstack([X_features, X_genre.toarray()])
y = df.rating
n_samples, n_features = X.shape

In [545]:
X

array([[ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.]])

In [546]:
n_samples

6384

In [547]:
n_features

873

# Content Filtering
Using this approach, I am creating a feature space where I am mapping the data to compare listeners to songs.
Each song gets a representation in the space and each listener gets a listener profile. That was it will become easy to recommend songs to a listener based on their profile

In [548]:
uniques, idx = np.unique(df.song_ids, return_index=True)
song_features = pd.DataFrame(X[idx, :], index=df.song_ids[idx])
print (song_features.shape)

(500, 873)


In [549]:
song_features.head(22)

,0,1,2,3,4,5,6,7,8,9,...,863,864,865,866,867,868,869,870,871,872
song_ids,,,,,,,,,,,,,,,,,,,,,
SOAAEHR12A6D4FB060,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
SOABRXK12A8C130A36,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
SOACUIU12A6D4F715B,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
SOAECHX12A6D4FC3D9,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SOAGXBV12A6D4F70BD,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
SOAHYMX12AB0182021,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
SOAIJKC12A8C131D6F,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
SOAOPCG12AC468D27E,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
SOAPIHC12A6D4F5FD5,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


# Listener Profile
Forming a profile for each listener which will be used later to recommend songs to the user.

In [550]:
listener_features = pd.DataFrame(X * (df.rating.values).reshape(n_samples, 1))
listener_features['listener_profile'] = df.user_id

In [552]:
listener_features = listener_features.groupby('listener_profile')[range(n_features)].mean()

In [553]:
listener_features = listener_features.divide(listener_features.sum(axis=1), axis=0)

In [554]:
listener_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6384 entries, 0 to 6383
Columns: 874 entries, 0 to listener_profile
dtypes: float64(873), object(1)
memory usage: 42.6+ MB


In [555]:
listener_features[[0]].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6384 entries, 0 to 6383
Data columns (total 1 columns):
0    6384 non-null float64
dtypes: float64(1)
memory usage: 50.0 KB


In [556]:
listener_features = listener_features.dropna(how='any')
listener_features.head(15)

,0,1,2,3,4,5,6,7,8,9,...,864,865,866,867,868,869,870,871,872,listener_profile
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,af031a4432b7a31431231a28459b04affbd0014e
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6c618f60b9b27b2b951854c56a7c96991514ca4f
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,06d61d9b33d47e83cfbd7be60f6191abaa600c21
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ffe70ee0afa3930edf9bca702cb3982d3ff747be
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18edc1032066fce485565100b646f36337547d0f
5,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,f4c64e7c47d5b99fe8b5e80ffdf88d162ecfb822
6,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5bb7e4903279ef8ddb3115e9bd359ca80ad3ec66
7,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,8035a56d59f07ea7388ab32465015649fc7a0e51
8,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,f3ebc843f7319ed08450553b048da3bedec18d5c
9,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,8463ade505658a6f70e9cfaa948b3b8515e18072


In [557]:
df.user_id

0       af031a4432b7a31431231a28459b04affbd0014e
1       6c618f60b9b27b2b951854c56a7c96991514ca4f
2       06d61d9b33d47e83cfbd7be60f6191abaa600c21
3       ffe70ee0afa3930edf9bca702cb3982d3ff747be
4       18edc1032066fce485565100b646f36337547d0f
5       f4c64e7c47d5b99fe8b5e80ffdf88d162ecfb822
6       5bb7e4903279ef8ddb3115e9bd359ca80ad3ec66
7       8035a56d59f07ea7388ab32465015649fc7a0e51
8       f3ebc843f7319ed08450553b048da3bedec18d5c
9       8463ade505658a6f70e9cfaa948b3b8515e18072
10      af1330e3d71a145538f586053b698ddee9228408
11      bb25adc62bedda82437af1fbe61cfad84ccc69a5
12      425a75a0bb2235f0b74aed542e50e4430bf2731d
13      a5602c5c3ca38815856b4d4ea15a6882f3496c5f
14      a6dc94c130c235a44eeca1d87bbc4d7a0a836e2c
15      c39fbe158c5cde44c1d1ce4a5031d736e73b2e63
16      524f171682ad00bf11b77bfa720f58147e6d189b
17      a16f207f75ff246b638b3a9120ee95a0caf0926e
18      970b8ce1c11b5d755050a36a690b6a72454a3ec6
19      cf7f7b9d0a4cc9b488e16c871eaac1e00bec02c8
20      402285aa65b2

In [558]:
df.user_id.value_counts().sort_index().head()

000724134199acaa98869fc4ffb805198d2940eb    1
001a4329228373ec5d4efe1084317b4009df7b97    1
0029b299091e9e9040761838785a0689a5f67215    1
002ace7725512ef9180af3c8a0fbf0e1071b72d9    2
002d1f1a39282b2875c08587379d9acf41866d57    1
Name: user_id, dtype: int64

In [559]:
user='001a4329228373ec5d4efe1084317b4009df7b97'
# v = listener_features.loc[:,user].values
v=listener_features.loc[listener_features['listener_profile']==user]
vv=v.copy()
v=v.drop(['listener_profile'],1)
print(v.shape)
very=v.values
# print(very)
Transpose=v.T
print(Transpose.shape)

(1, 873)
(873, 1)


In [560]:
M = song_features.values
print(M.shape)
print(Transpose.shape)
pred = M.dot(Transpose)
predcopy=pred
predd=predcopy.flatten()
print(predd.shape)

(500, 873)
(873, 1)
(500,)


In [561]:
# Now index this vector so we can understand the values and use them to make recommendations
from pandas import Series
predd = pd.Series(predd, index=song_features.index, name="predictions").sort_values(ascending=False, inplace=False)

In [562]:
pred_name = pd.Series(predd.values, song_names[predd.index], name=predd.name)

In [563]:
print("Top recommendations for %s" % user)
print(pred_name.head())

Top recommendations for 001a4329228373ec5d4efe1084317b4009df7b97
song_titles
Crawling (Album Version)    8.0
Alpha Beta Parking Lot      4.0
Supreme Girl                4.0
Is It A Myth?               4.0
Too Shy To Scream           4.0
Name: predictions, dtype: float64


In [564]:
print(pred_name)

song_titles
Crawling (Album Version)             8.0
Alpha Beta Parking Lot               4.0
Supreme Girl                         4.0
Is It A Myth?                        4.0
Too Shy To Scream                    4.0
One Night Alone                      4.0
Everybodys Gonna Be Happy            4.0
Overdrive                            4.0
Who Did You Think I Was              4.0
Get Away                             4.0
Sensory Deprivation Adventure        4.0
I Never Came                         4.0
All My Senses                        4.0
Desde mi cielo                       4.0
City Headache                        4.0
Grandbags Funeral                    4.0
Intro/Chamber The Cartridge          4.0
Bad Seed                             4.0
Sahti Waari                          4.0
Sloppy (I Saw My Baby Gettin)        4.0
Puzzle of Hearts                     4.0
Won Threw Ate                        4.0
Roman Empire                         4.0
Crazy                                4.0
Part

# Comparison
Let us compare the above result to the actual reviews and see if our model's output matches or not.

In [565]:
my_reviews = df[df.user_id == user].groupby('song_ids').rating.mean().sort_values(ascending=False, inplace=False)

In [566]:
my_reviews_name = pd.Series(my_reviews.values, index=song_names[my_reviews.index])

In [567]:
print ("Top reviewed by %s" % user)
print (my_reviews_name.head())

Top reviewed by 001a4329228373ec5d4efe1084317b4009df7b97
song_titles
Crawling (Album Version)    2
dtype: int64


In [568]:
print ("\nBottom reviewed by %s" % user)
print (my_reviews_name.tail())


Bottom reviewed by 001a4329228373ec5d4efe1084317b4009df7b97
song_titles
Crawling (Album Version)    2
dtype: int64


# Note-
From the above results it is clear that our model recommends the right songs to the user. Its first recommendation is same as based on song reviews. (Crawling (Album Version))

# Collaborative Filtering
This seems to be a perfect fit when it comes to making recommendations to a user. Let us try this apporach as well and see how my model works.

# 1. Item-based collaborative filtering

In [576]:
# Here, I am creating a song x listener preference matrix
M2 = df.groupby(['song_ids','user_id']).rating.mean().unstack()
M2 = np.maximum(-1, (M2 - 3).fillna(0) / 2.)  # scale to -1..+1  (treat "0" scores as "1" scores)
M2.head(2)

user_id,000724134199acaa98869fc4ffb805198d2940eb,001a4329228373ec5d4efe1084317b4009df7b97,0029b299091e9e9040761838785a0689a5f67215,002ace7725512ef9180af3c8a0fbf0e1071b72d9,002d1f1a39282b2875c08587379d9acf41866d57,003032fc00365d6f15f047c63717a14e0d8aced7,00368ec2f1ed8496ea085de2b338c314b6aa2c06,00539b30daa6999e65cc44867acac6c336004a02,00545af48096210f97930b66f6661df88729900f,00692118f3481898a924c22488ae5600a55249f6,...,ff9c5a4fdbbe94eb70098ee066e5fd84c4ee3895,ffa9ab7e3ff7b5d4fbac66bc9997949a9e721a91,ffbada23cac620140e67e9d473e6d9bd46467973,ffc1792e10684eedde8364f1bd875c781d887086,ffe33c42d3c85ad750326fa0ae45cc3b84d58f15,ffe5a0afac73eefdc4e6cfb0a40df22480ed4ce2,ffe70ee0afa3930edf9bca702cb3982d3ff747be,ffec87ea10f29a7103edd660c45c65527b1c4b9d,ffee916020d891a74264b8326829742333b79103,fffcfe9b89b791e39ff3e643aa57ae9303079c56
song_ids,,,,,,,,,,,,,,,,,,,,,
SOAAEHR12A6D4FB060,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SOABRXK12A8C130A36,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [580]:
n_ratings = data.groupby(['song_ids', 'user_id']).size()

In [574]:
# keeping 1 for user-song rating
n_ratings[n_ratings  > 1] = 1

In [582]:
# creating matrix and then fill empty cells with 0
n_ratings = n_ratings.unstack().fillna(0)

In [462]:
n_ratings = n_ratings.dot(n_ratings.T)  # how many ratings do two identities have in common?
n_ratings

song_ids,SOAAEHR12A6D4FB060,SOABRXK12A8C130A36,SOACUIU12A6D4F715B,SOAECHX12A6D4FC3D9,SOAGXBV12A6D4F70BD,SOAHYMX12AB0182021,SOAIJKC12A8C131D6F,SOAOPCG12AC468D27E,SOAPIHC12A6D4F5FD5,SOAPTWU12A8C1409CB,...,SOZCURA12AF72A17B1,SOZDDPQ12AF72A8BA8,SOZFZMX12AB018BE08,SOZLSBK12A6D4F87B8,SOZMELK12A8C143532,SOZQDRV12A8C133FB1,SOZSKYN12A8C13A883,SOZVTZB12A58A7AEE4,SOZXHBQ12AB0186626,SOZXTKD12A8C13FC43
song_ids,,,,,,,,,,,,,,,,,,,,,
SOAAEHR12A6D4FB060,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SOABRXK12A8C130A36,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SOACUIU12A6D4F715B,0.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SOAECHX12A6D4FC3D9,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SOAGXBV12A6D4F70BD,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SOAHYMX12AB0182021,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SOAIJKC12A8C131D6F,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SOAOPCG12AC468D27E,0.0,0.0,0.0,0.0,0.0,0.0,0.0,142.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SOAPIHC12A6D4F5FD5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [463]:
# Here, we are going to ignore the songs that have less ratings
ignore_up_to_n = 3
n_ratings = np.maximum(ignore_up_to_n, n_ratings).replace(ignore_up_to_n, np.nan)

In [464]:
# Calculating the score in common between two songs
sim = M.dot(M.T)
sim = sim / n_ratings 

In [465]:
sim.head(4)

song_ids,SOAAEHR12A6D4FB060,SOABRXK12A8C130A36,SOACUIU12A6D4F715B,SOAECHX12A6D4FC3D9,SOAGXBV12A6D4F70BD,SOAHYMX12AB0182021,SOAIJKC12A8C131D6F,SOAOPCG12AC468D27E,SOAPIHC12A6D4F5FD5,SOAPTWU12A8C1409CB,...,SOZCURA12AF72A17B1,SOZDDPQ12AF72A8BA8,SOZFZMX12AB018BE08,SOZLSBK12A6D4F87B8,SOZMELK12A8C143532,SOZQDRV12A8C133FB1,SOZSKYN12A8C13A883,SOZVTZB12A58A7AEE4,SOZXHBQ12AB0186626,SOZXTKD12A8C13FC43
song_ids,,,,,,,,,,,,,,,,,,,,,
SOAAEHR12A6D4FB060,0.272727,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SOABRXK12A8C130A36,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SOACUIU12A6D4F715B,NaN,NaN,0.222222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SOAECHX12A6D4FC3D9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


 # Note-
If we have no common ratings between 2 songs, we get a NaN now.

In [486]:
n_ratings = data.groupby(['song_ids']).size()
print(n_ratings)
songs = n_ratings[n_ratings > 10].index

song_ids
SOAAEHR12A6D4FB060     11
SOABRXK12A8C130A36      5
SOACUIU12A6D4F715B     18
SOAECHX12A6D4FC3D9      3
SOAGXBV12A6D4F70BD      2
SOAHYMX12AB0182021      2
SOAIJKC12A8C131D6F      1
SOAOPCG12AC468D27E    142
SOAPIHC12A6D4F5FD5      3
SOAPTWU12A8C1409CB      1
SOAPZIG12A6D4F7920      1
SOARSSE12A58A7BA9A      6
SOATHUP12A8C131DC3      2
SOAUMHT12AB01839D0      2
SOAUXQN12AB018D2F6      7
SOAVGKZ12A81C1FD16     11
SOAWVZG12A8C1443A5      5
SOAXBAE12A6D4F79C3     12
SOAXMZC12A6D4F67A6      4
SOAYOFO12AF72A4B88     15
SOAYXCJ12AB0183822     13
SOAZSIZ12AB018AF81      2
SOAZWHG12AB01861BA      1
SOBDLAF12AF72A69D7      1
SOBFTEV12A8C13B4A4      2
SOBFZKC12A6D4FC1F7     25
SOBGPAH12A58A78C55      6
SOBHHUS12A58A78589     10
SOBKDCU12A6D4F80F6      6
SOBMNXE12A8C140F8A     89
                     ... 
SOYCOEX12AB018B5DF      3
SOYCTRU12A6701DA91      4
SOYDTJN12AF72A86B9      1
SOYDZUT12A8C13677D      1
SOYGOKZ12A58A7C65D      2
SOYHZWP12A58A7C176      1
SOYIXMY12AF72A0734     14
SOY

In [487]:
n_songs = 5
top = 5
for song in np.random.choice(songs, n_songs):
    print("The song name is-", song_names[song])
    sim_songs = sim[song].sort_values(ascending=False, inplace=False).dropna()
    if len(sim_songs) == 0:
        print("Not enough comparisons")
        print('')
    else:
        print("MOST  similar:", ", ".join(song_names[list(sim_songs.head(top).index)]))
        print("LEAST similar:", ", ".join(song_names[list(sim_songs.tail(top).index)]))
        print('')

The song name is- The Earth Will Shake
MOST  similar: The Earth Will Shake
LEAST similar: The Earth Will Shake

The song name is- U Killed It
MOST  similar: U Killed It
LEAST similar: U Killed It

The song name is- Decimas
MOST  similar: Decimas
LEAST similar: Decimas

The song name is- You Can Do Better Than Me (Album Version)
MOST  similar: You Can Do Better Than Me (Album Version)
LEAST similar: You Can Do Better Than Me (Album Version)

The song name is- In My Arms
MOST  similar: In My Arms
LEAST similar: In My Arms



In [468]:
print("Top reviews for %s are" % user)
print(my_reviews_name.head())

Top reviews for 001a4329228373ec5d4efe1084317b4009df7b97 are
song_titles
Crawling (Album Version)    2
dtype: int64


In [483]:
favorite_songs = list(my_reviews[my_reviews > 1].index)
pred = sim[favorite_songs].mean(axis=1).dropna().sort_values(ascending=False, inplace=False)
pred_name = pd.Series(pred.values, song_names[pred.index], name=pred.name)
print("Top recommendations for %s are" % user)
print(pred_name.head())
print("\nBottom recommendations for %s are" % user)
print(pred_name.tail())

Top recommendations for 001a4329228373ec5d4efe1084317b4009df7b97 are
song_titles
Crawling (Album Version)    0.008475
dtype: float64

Bottom recommendations for 001a4329228373ec5d4efe1084317b4009df7b97 are
song_titles
Crawling (Album Version)    0.008475
dtype: float64


# Discussion
From the above results obtained, we can conclude on the following things-
1. Feature selection using RFE and Extra tree classifier worked well for us.
2. Supervised learning methods did not work well for our model.
3. Unsupervised learning method such as k-means clustering seems to work better than supervised learning methods.
4. Content filtering, collaborative filtering methods are used to make song recommendations to the user based on the listener profile that is created first.
5. Both the actual reviews and item-based CF seems to be working well. They provide exact same song recommendations to the user based on their profile which forms the transparent part of this project.
6. Future work would be to use the same songs recommended to the user, go to their lyrics and predict if it is a happy or a sad song based on words hand picked by a human labeller.
Part of the future work is done while I was preprocessing dat is shown below where I check for the recommended song, read the human labelled csv files containing positive or negative words, search for them in the lyrics column and predict the song type as happy or sad.

# Future work

# 
Words.csv is a csv file that is made by a human labeller who hand picked positive and negative words at random.

In [2]:
df_words=pd.read_csv('words.csv',delimiter=',')
df_words

,Postive words,Negative words
0,advisable,sad
1,adventures,gloomy
2,affable,death
3,affection,died
4,spread,dying
5,joy,sadness
6,happiness,crying
7,happy,cri
8,laugh,cry
9,laughter,hatred


In [5]:
D2=pd.read_csv('Final.csv',sep=',')

,Unnamed: 0,artist_familiarty,artist_hotness,artist_names,song_bars,song_beats,song_durations,song_hotness,song_ids,song_keys,...,song_tempo,song_time_signatures,song_titles,track_id,genre,lyrics,artistlocation,user_id,play_count,rating
0,0,0.733433,0.455559,Shadows Fall,[ 1.35712 4.17899 7.00987 9.83158 ...,[ 0.65324 1.35712 2.06418 2.76881 ...,283.48036,0.687874,SOKMPKV12A67AE241B,0,...,84.992,4,What Drives The Weak,TRARUOP12903CF2384,Metal,the|19 you|3 to|6 a|4 me|1 it|4 not|1 in|1 is|...,Boston,af031a4432b7a31431231a28459b04affbd0014e,1,1
1,1,0.733433,0.455559,Shadows Fall,[ 1.35712 4.17899 7.00987 9.83158 ...,[ 0.65324 1.35712 2.06418 2.76881 ...,283.48036,0.687874,SOKMPKV12A67AE241B,0,...,84.992,4,What Drives The Weak,TRARUOP12903CF2384,Metal,the|19 you|3 to|6 a|4 me|1 it|4 not|1 in|1 is|...,Boston,6c618f60b9b27b2b951854c56a7c96991514ca4f,1,1


In [8]:
song_names = D2.groupby('track_id').song_titles.first()
song_df=D2.groupby('track_id')['song_titles','artist_names'].first()
song_df.head(3)

,song_titles,artist_names
track_id,,
TRAAABD128F429CF47,Soul Deep,The Box Tops
TRAABVM128F92CA9DC,Caught In A Dream,Tesla
TRAADQX128F422B4CF,Nashville Parthenon,Casiotone For The Painfully Alone


In [11]:
D2.rating.value_counts().sort_index()

1    4783
2    1332
3     188
4      55
5      26
Name: rating, dtype: int64

In [12]:
D2.genre.value_counts().sort_index()

Blues                   47
Blues|Rock             127
Country                157
Country|Folk             3
Country|Rock             6
Electronic             157
Electronic|Pop          12
Electronic|Punk          7
Electronic|Rock          5
Folk                     2
Folk|Country             7
Folk|Pop                16
Jazz                    12
Jazz|Rock               10
Latin                   55
Latin|Pop               11
Latin|Reggae             5
Latin|Rock              37
Latin|World              2
Metal                   29
Metal|Punk               2
Metal|Rock              19
New Age|Electronic       2
Pop                    305
Pop|Electronic          21
Pop|Jazz                 1
Pop|Latin              118
Pop|Rap                211
Pop|Rock               155
Punk                   167
Punk|Rock               19
Rap                    364
Rap|Electronic          16
Rap|RnB                  2
Rap|Rock                 6
Reggae                 122
Reggae|Pop               5
R

In [13]:
list_lyrics=D2['lyrics'].tolist()
song_names=D2['song_titles'].tolist()
copy_df_lyrics=[]
for k in list_lyrics:
    k = re.sub(r'\|', '',  k)
    if k and k not in copy_df_lyrics:
        copy_df_lyrics.append(k)
lyrics_live=[]
for i in copy_df_lyrics:
    j = i.replace(' ','')
    lyrics_live.append(j)
print(lyrics_live)

['the19you3to6a4me1it4not1in1is4of5on3are3all3but1with1what4time1come3as2yeah1night2world1onli2mind1an2over1gone3place1anoth2put4sleep5death1forget1voic1control1rest1drive4becom2near1within1weak4remain2wash1yesterday2insan1state1releas1heartbeat1secur1skip1panic7alter1', 'i13the2to4a2me3it1not6in6my8is13of4that4are2am2will5have2so2this4what6just2there4oh1they4way2take2ca2how5again2too2find2fall4has1someth2insid2befor2much2stand1these4without2seem2own1myself2close2real6fear6wall2control2upon1pull3skin4felt2against1sens2crawl4endless1heal4beneath2confus8wound4besid1haunt1reflect1pressur2consum2lack2itself1surfac2confid2convinc2insecur2distract1', 'i10the1you13and3a1me15it1not7in1my14is1do8are2we3will7all1love2so1know2but1now12there1oh2one1get1see1got6want2make1babi7yeah8where2thing1more1right11tell2think2again2mind2girl6wanna1over1then2leav5everyth2chang2pleas8togeth1ah2sometim2ha1citi1fine1meet1honey1darl1babe1satisfi2', 'i13the1you15to2and1a7not3in14my3of2your13that4do4on1am5will2all1f

In [39]:
df_pwords=df_words['Postive words'].tolist()
df_nwords=df_words['Negative words'].tolist()

In [67]:
pclss=[]
pwords=[]
nwords=[]
for y in lyrics_live:
    for j in df_pwords:
        count=0
        if j in y:
            count=count+1
            if count==1:
                pclss.extend([count])
                pwords.extend([j])
                break
        elif j not in y:
            pclss.extend([count])
            for a in df_nwords:
                count=0
                if a in y:
                    count=count+1
                    if count==1:
                        nwords.extend([j])
                        break

In [70]:
pwords

['heart',
 'find',
 'love',
 'chang',
 'ark',
 'love',
 'life',
 'pray',
 'love',
 'laugh',
 'born',
 'care',
 'babi',
 'babi',
 'laugh',
 'ark',
 'life',
 'love',
 'ark',
 'best',
 'love',
 'light',
 'free',
 'life',
 'best',
 'god',
 'smile',
 'light',
 'faith',
 'pray',
 'best',
 'smile',
 'light',
 'care',
 'like',
 'best',
 'light',
 'talkin',
 'danc',
 'ark',
 'light',
 'faith',
 'love',
 'like',
 'love',
 'smile',
 'smile',
 'laugh',
 'best',
 'heart',
 'friend',
 'ark',
 'light',
 'real',
 'find',
 'love',
 'love',
 'light',
 'love',
 'smile',
 'care',
 'chang',
 'love',
 'love',
 'wonder',
 'love',
 'love',
 'ark',
 'smile',
 'love',
 'laugh',
 'smile',
 'smile',
 'love',
 'care',
 'love',
 'light',
 'love',
 'love',
 'heart',
 'care',
 'joy',
 'light',
 'love',
 'chang',
 'light',
 'smile',
 'find',
 'faith',
 'love',
 'care',
 'love',
 'light',
 'smile',
 'perfect',
 'love',
 'care',
 'faith',
 'love',
 'smile',
 'care',
 'spread',
 'smile',
 'like',
 'pray',
 'love',
 'smil

# Discussion
From the above results obtained, we can see that if the positive word is present in lyrics, we get the pwords along with pclass and vice versa for negative words. These can then be added to the actual df using a new column called class where 0 is happy 1 is negative and then we can use that to train our model to predict the song mood for that song recommended to the user.
